In [8]:
from multiprocessing import Pool

In [9]:
# parallelize function
def product(a,b):
    print a*b

In [10]:
# auxiliary funciton to make it work
def product_helper(args):
    return product(*args)

In [11]:
def parallel_product(list_a, list_b):
    # spark given number of processes
    p = Pool(5) 
    # set each matching item into a tuple
    job_args = [(item_a, list_b[i]) for i, item_a in enumerate(list_a)]
    print job_args
    # map to pool
    p.map(product_helper, job_args)

In [12]:
exp_a = range(1000)
exp_b = range(1000)
parallel_product(exp_a, exp_b)

0
10000
1
2500
22500
10201
40000
22801
4
40401
10404
23104
40804
10609
2601
23409
9
41209
2704
23716
16
10816
41616
2809
24025
11025
2916
25
11236
42025
36
3025
42436
49
24336
42849
11449
64
24649
81
43264
24964
11664
100
43681
3136
121
44100
11881
144
44521
3249
3721
12100
169
25281
3364
44944
12321
196
3481
25600
25921
3600
3844
12544
225
45796
26244
46225
12769
45369
46656
3969
26569
289
12996
256
13225
4096
47089
26896
47524
324
13456
47961
27225
361
4225
48400
27556
4356
4489
400
13689
48841
441
49284
4624
27889
13924
49729
484
50176
4761
28224
529
28561
50625
4900
28900
14161
5041
51076
29241
576
14400
29584
51529
5184
625
14641
51984
5329
29929
52441
14884
30276
15129
5476
52900
676
30625
5625
15376
53361
30976
5776
15625
729
53824
15876
31329
5929
16129
6084
6241
784
31684
16384
841
32041
16641
900
32400
961
32761
16900
6400
1024
33124
6561
1089
54289
1156
17161
6724
54756
33489
1225
17424
1296
33856
6889
55225
1369
34225
7056
34596
17689
1444
7225
7396
34969
55696
1521
7569
35

In [14]:
for i in range(1,8):
    print i

1
2
3
4
5
6
7
